# Optimization of a Basket of Goods.

Say we have 10 foods, listed below, and we know the average amount of FOLATE, BETA-CAROTENE, and ASCORBIC ACID (each of these build vital nutrients) they contain per 100mg.

In [1]:
#[[FOLATE, BETA-CAROTENE, ASCORBIC ACID]]
foods = [
    [0.06445, 0.91988, 35.580], # Apple
    [8.403, 461.750, 30.000],   # Swiss Chard
    [19.062, 0.023, 33.464],    # Cauliflower
    [15.245, 1.191, 13.125],    # Turnip
    [0.00800, 0.42, 27.000],    # Persimmon
    [0.20340, 0.08350, 28.067], # Cranberry
    [0.02813, 14.477, 7.345],   # Carrot
    [91.960, 4.307, 1.655],     # Chickpea
    [67.023, 2.257, 5.990],     # Lentils
    [6.5, 3.725, 136.033],      # Onion
]

We have five astronauts, and based upon their gene expressions, they are either deficient or proficient in proccessing the above three compounds.

In [1]:
DEFICIENT  = .75
PROFICIENT = 1.0

astronaut_1 = [DEFICIENT, PROFICIENT, PROFICIENT]
astronaut_2 = [PROFICIENT, DEFICIENT, DEFICIENT]
astronaut_3 = [PROFICIENT, DEFICIENT, PROFICIENT]
astronaut_4 = [DEFICIENT, PROFICIENT, DEFICIENT]
astronaut_5 = [PROFICIENT, PROFICIENT, PROFICIENT]

astronauts = astronaut_1 + astronaut_2 + astronaut_3 + astronaut_4 + astronaut_5

In [3]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd

Our question thus becomes how much of each food do we need to send to space to satisfy each of these astronauts dietary needs.
Each astronaut can be allotted some number of each food; this totals to 50 different amounts of food (`5 astronauts * len(foods)).

In [4]:
model = pyo.ConcreteModel()
model.x = pyo.Var([x for x in range(0, 5 * len(foods))], domain=pyo.NonNegativeReals)

We want to minimize the amount of food spent to space.

In [5]:
model.OBJ = pyo.Objective(expr = pyo.summation(model.x))

Each astronaut has a constraint: the sum of the foods they eat must be larger than their daily dietary needed taking into account their genetic profile that influences their metabolism.

In [6]:
model.C = pyo.ConstraintList()

def add_constraint_dv(a_idx, v_idx, dv):
    """ Add a constraint that for astronaut a_idx, they must receive
    dv of the v_idx of each food. """
    
    lb = a_idx * len(foods)
    ub = lb + 10
    
    s = 0
    # x_subset = model.x[lb:ub] # I can't slice this apparently, FML.
    #for (x_j, f_j) in zip(x_subset, foods):
    #    s += x_j * f_j[v_idx]
    s += model.x[lb]     * foods[0][v_idx] + model.x[lb + 1] * foods[1][v_idx] + model.x[lb + 2] * foods[2][v_idx] 
    s += model.x[lb + 3] * foods[3][v_idx] + model.x[lb + 4] * foods[4][v_idx] + model.x[lb + 5] * foods[5][v_idx]
    s += model.x[lb + 6] * foods[6][v_idx] + model.x[lb + 7] * foods[7][v_idx] + model.x[lb + 8] * foods[8][v_idx]
    s += model.x[lb + 9] * foods[9][v_idx]
    
    model.C.add(expr = astronauts[a_idx] * s >= dv)

In [7]:
for i in range(0, 5):
    add_constraint_dv(i, 0, 400) # Each astronauts DV for folate.
    add_constraint_dv(i, 1, 300) # Each astronauts DV for Beta-Carotene
    add_constraint_dv(i, 2, 90)  # Each astronauts DV for Ascorbic Acid

We can solve this linear programming problem and print out a solution that minimizes the total amount of food needed while satisfying the dietary requirement. Each basket (x0 -> x9, x11 -> x20, etc.) corresponds to the allotted amount of those 10 foods for each astronaut.

In [8]:
def print_solution(model):
    # TODO: HACK
    outer = []
    for i in range(0, 5):
        inner = []
        for j in range(0, 10):
            inner.append(model.x[i * 10 + j].value)
        outer.append(inner)

    df = pd.DataFrame(outer, columns=["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9", "F10"])
    df.insert(0, "Name", ["Astro1", "Astro2", "Astro3", "Astro4", "Astro5"])

    print(df)
            
    #for i in range(0, 5 * len(foods)):
    #    print("x{}: {}".format(i, model.x[i].value))

In [9]:
opt = SolverFactory("glpk")
result = opt.solve(model)

# for i in range(0, 5 * len(foods)):
#    print("x{}: {}".format(i, model.x[i].value))

print_solution(model)

     Name   F1       F2   F3   F4   F5   F6   F7        F8   F9       F10
0  Astro1  0.0  0.80816  0.0  0.0  0.0  0.0  0.0  5.680907  0.0  0.634797
1  Astro2  0.0  0.60612  0.0  0.0  0.0  0.0  0.0  4.260680  0.0  0.476098
2  Astro3  0.0  0.60612  0.0  0.0  0.0  0.0  0.0  4.260680  0.0  0.476098
3  Astro4  0.0  0.60612  0.0  0.0  0.0  0.0  0.0  4.260680  0.0  0.476098
4  Astro5  0.0  0.80816  0.0  0.0  0.0  0.0  0.0  5.680907  0.0  0.634797


We can add other constraints however; say the last astronaut won't eat onion (x49). As you can see, solving for this constraint produces
a different x49 value than the previous solution (0.6347... vs 0.0).

In [10]:
model.C.add(expr = model.x[49] == 0.0)
result = opt.solve(model)

print(model.x[49].value)

0.0


Another constraint might be that the total number of apples (x0, x10, ...) sent for the astronauts as a whole has to be larger than 3. Again, solving for this constraint produces a different solution.

In [11]:
model.C.add(expr = model.x[0] + model.x[10] + model.x[20] + model.x[30] + model.x[40] >= 3)
result = opt.solve(model)

print_solution(model)

#for i in range(0, 5 * len(foods)):
#    print("x{}: {}".format(i, model.x[i].value))

     Name        F1        F2   F3   F4   F5   F6   F7        F8   F9  \
0  Astro1  0.574895  0.808133  0.0  0.0  0.0  0.0  0.0  5.691143  0.0   
1  Astro2  0.000000  0.606120  0.0  0.0  0.0  0.0  0.0  4.260680  0.0   
2  Astro3  0.000000  0.606120  0.0  0.0  0.0  0.0  0.0  4.260680  0.0   
3  Astro4  0.000000  0.606120  0.0  0.0  0.0  0.0  0.0  4.260680  0.0   
4  Astro5  2.425105  0.808047  0.0  0.0  0.0  0.0  0.0  5.724087  0.0   

        F10  
0  0.484312  
1  0.476098  
2  0.476098  
3  0.476098  
4  0.000000  


These are simply silly linear constaints but they off a powerful platform; one can build constraints and models that take into account price, weight, area used, specific tastes, etc. More complicated models might take into account half-life decay of nutrients over a period of time, or add random variables to simulate spoilage (solved through robust optimization!). Another point to consider is that these are real values; what is .03320301934 apples? Further work could be done to discretize the optimization.

In [12]:
print(model.x[1].value)

0.808132865224022
